In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
DATASET_PATH = "data.json"

In [ ]:
def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
        
        inputs = np.array(data["mfcc"])
        targets = np.array(data["labels"])
        
        return inputs, targets
    
inputs, targets = load_data(DATASET_PATH)


inputs_train , inputs_test, targets_train, targets_test = train_test_split(inputs, targets, test_size=0.3) 

In [ ]:
model = keras.Sequential([
    #input layer
    keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),
    
   
    
    #1st hidden layer
    keras.layers.Dense(512, activation="relu"),

    #2nd hidden layer
    keras.layers.Dense(256, activation="relu"),
    
    #3rd hidden layer
    keras.layers.Dense(64, activation="relu"),
    
    #output layer
    keras.layers.Dense(2, activation="softmax")
])


In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()


In [ ]:
model.fit(inputs_train, targets_train, validation_data=(inputs_test, targets_test), epochs=50, batch_size=32)


In [ ]:
model.save('tipe.model')

In [ ]:
label_encoder = LabelEncoder()
encoded_targets_test = label_encoder.fit_transform(targets_test)

# Compute the confusion matrix
cm = confusion_matrix(encoded_targets_test, np.argmax(model.predict(inputs_test), axis=1), labels=[0, 1])

# Define the class labels for plotting
classes = label_encoder.classes_

# Define a function to plot the confusion matrix
def plot_confusion_matrix(cm, classes, title='Matrice de confusion'):
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('Données Réelles')
    plt.xlabel('Données prédites')
    plt.tight_layout()

# Plot the confusion matrix
plot_confusion_matrix(cm, classes)
plt.show()
